In [4]:
import torch
import torch_geometric.utils as tg
#from torch_geometric.transforms import remove_duplicated_edges
from rephine_mt import compute_rephine_batched_mt
from spectre import compute_spectre_batched_mt
from ph_cpu import compute_persistence_homology_batched_mt
import numpy.linalg as linalg

import collections


In [5]:
def compute_persistence(diagram_type, x, e, edge_index, vertex_slices, edge_slices):
    filtered_v = x
    filtered_e = e
    if diagram_type == "rephine":
        compute = compute_rephine_batched_mt
    elif diagram_type == "spectre":
        compute = compute_spectre_batched_mt
    elif diagram_type == "standard":
        filtered_e, _ = torch.max(
            torch.stack((filtered_v[edge_index[0]], filtered_v[edge_index[1]])),
            axis=0,
        )
        compute = compute_persistence_homology_batched_mt
    elif diagram_type == "edge":
        compute = compute_rephine_batched_mt

    vertex_slices = vertex_slices.cpu().long()
    edge_slices = edge_slices.cpu().long()
    filtered_v = filtered_v.transpose(1, 0).cpu().contiguous()
    filtered_e = filtered_e.transpose(1, 0).cpu().contiguous()
    edge_index = edge_index.cpu().transpose(1, 0).contiguous()

    
    persistence0, persistence1 = compute(
        filtered_v, filtered_e, edge_index, vertex_slices, edge_slices
    )

    persistence0 = persistence0.to(x.device)
    persistence1 = persistence1.to(x.device)

    if diagram_type == "rephine" or diagram_type == 'spectre' or diagram_type == 'edge':
        full_size = persistence0.shape[2]
        indices = list(range(3, full_size, 1))
        persistence0 = persistence0[:, :, [0, 2, 1] + indices]

        persistence0 = torch.cat(
            (
                torch.zeros((persistence0.shape[0], persistence0.shape[1], 1)).to(
                    x.device
                ),
                persistence0,
            ),
            dim=-1,
        )
        persistence1 = torch.cat(
            (
                torch.zeros((persistence1.shape[0], persistence1.shape[1], 1)).to(
                    x.device
                ),
                persistence1,
            ),
            dim=-1,
        )

        persistence0[persistence0.isnan()] = 1000

    return persistence0.squeeze(), persistence1.squeeze()

def hks_signature(eigenvectors, eigenvals, time=1):
    return np.square(eigenvectors).dot(np.diag(np.exp(-time * eigenvals))).sum(axis=1)


In [6]:
seed_dataset = 42
choices = [
#    "minCayleyGraphs12Vertices", 
#    "minCayleyGraphs16Vertices",
#    "minCayleyGraphs20Vertices", 
     "minCayleyGraphs24Vertices",
#    "minCayleyGraphs32Vertices", 
#    "srminCayleyGraphs36Vertices",
#    "srminCayleyGraphs60Vertices", 
#    "minCayleyGraphs63Vertices"
]

diagrams = ['standard', 'rephine', 'spectre', 'edge']
for setting in choices:
    file = f"./datasets/cayley/{setting}_seed-{seed_dataset}.dat"
    dataset = torch.load(file)

    results = {}
    j = 0
    for g in dataset:
        adj = tg.to_dense_adj(g.edge_index)
        degree = adj.sum(dim=-1).squeeze()
        g.x = degree.unsqueeze(dim=0).T

        correct_idx = g.edge_index[0] <= g.edge_index[1]
        new_edge_index = g.edge_index[:, correct_idx]
        edge_features = torch.zeros(new_edge_index.shape[1], 1)
        for i in range(new_edge_index.shape[1]):
            e = [g.edge_index[0, i].item(), g.edge_index[1, i].item()]
            edge_features[i] = 4 - degree[e[0]] - degree[e[1]]
            neighbors = g.edge_index[1, g.edge_index[0] == e[0]].tolist()
            neighbors2 = g.edge_index[1, g.edge_index[0] == e[1]].tolist()
            edge_features[i] += 3 * len(set(neighbors) & set(neighbors2))
        g.edge_features = edge_features
        g.edge_index = new_edge_index
        
        vs = torch.tensor([0, g.x.shape[0]]).long()
        es = torch.tensor([0, g.edge_index.shape[1]]).long()
        values = {}
        for diagram in diagrams:
            if diagram == 'edge':
                g.x = -1*torch.ones(degree.shape[0], 1)

            d10, d11 = compute_persistence(diagram_type=diagram, x=g.x, e=g.edge_features, 
                                           edge_index=g.edge_index, 
                                           vertex_slices=vs, edge_slices=es)
            if diagram == 'edge':
                d10=d10[:, :2]
                d11=d11[:, :2]
                
            values[diagram] = [d10, d11]
        results[j]=values
        j = j+1

    arr = torch.tensor(list(range(0, len(dataset))))
    indices = torch.combinations(arr, 2)
        
    for diagram in diagrams:
        count = 0
        for i in indices:
            a = i.tolist()
            d10, d11 = results[a[0]][diagram]
            d20, d21 = results[a[1]][diagram]
            
            l1 = list() 
            for b in range(d10.shape[0]):
                l1.append(tuple(d10[b].numpy().round(3)))

            l2 = list() 
            for b in range(d20.shape[0]):
                l2.append(tuple(d20[b].numpy().round(3)))
            
            l11 = list()
            for b in range(d11.shape[0]):
                l11.append(tuple(d11[b].numpy().round(3)))

            l21 = list() 
            for b in range(d21.shape[0]):
                l21.append(tuple(d21[b].numpy().round(3)))
            
            if (collections.Counter(l1) == collections.Counter(l2)) and (collections.Counter(l11) == collections.Counter(l21)):
                count+=1
        print(f'{setting}, {diagram}:, {(1.0-count/len(indices)):.2f}')

NameError: name 'diagram' is not defined